# *Using VGG16 pretrained model to classify Concrete Crack images*

## Import Libraries and Packages

In [1]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense

## Define Global Constants

Here, we will define constants that we will be using throughout the rest of the lab.

1.  We are obviously dealing with two classes, so *num_classes* is 2.
2.  The ResNet50 model was built and trained using images of size (224 x 224). Therefore, we will have to resize our images from (227 x 227) to (224 x 224).
3.  We will training and validating the model using batches of 100 images.

In [2]:
num_classes = 2
image_resize = 224
batch_size_training = 100
batch_size_validation = 100

## Construct ImageDataGenerator Instances


In [3]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [4]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_2/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [5]:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_2/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 10001 images belonging to 2 classes.


## Build, Compile and Fit Model


In [6]:
model = Sequential()
model.add(VGG16(include_top=False,pooling='avg',weights='imagenet'))


In [7]:
model.add(Dense(num_classes, activation='softmax'))
model.layers


In [8]:
model.layers[0].layers

In [9]:
model.layers[0].trainable = False

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense (Dense)               (None, 2)                 1026      
                                                                 
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [13]:
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

/var/folders/ks/m0btnx4d7j38phjw29dfybvw0000gn/T/ipykernel_5241/3326332652.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fit_history = model.fit_generator(
2022-08-17 12:23:20.705699: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/2
301/301 [==============================] - 3698s 12s/step - loss: 0.0880 - accuracy: 0.9741 - val_loss: 0.0269 - val_accuracy: 0.9942
Epoch 2/2
301/301 [==============================] - 3725s 12s/step - loss: 0.0215 - accuracy: 0.9949 - val_loss: 0.0160 - val_accuracy: 0.9959


#### Model Saving

In [14]:
model.save('classifier_VGG16_model.h5')